# Black magic with regular expressions

An incredibly abbreviated overview of regex:

- Regular Expressions are a mini-language for describing strings
- Usually used to check if a string matches some pattern, or extract part of a structured string

Interesting tidbits:

- Regular expressions\* are equivalent to finite state machines
- We can therefore 'run it backwards' to spit out strings that would match!


<small>*Group references and look-around assertions can't be computed in sub-exponential time, so 'PCRE' are not technically regular expressions.  But they're useful, so everyone handles them and risks DoS attacks anyway.*</small>

## Hang on, why do we care?

Because [Hypothesis](https://hypothesis.readthedocs.io) can now generate input strings for your tests using regular expressions.

The rest of this talk is about how this works:

- Hidden regex tools in the standard library
- How we compile a regex parse tree into a Hypothesis strategy
- Optimisations we have - and haven't - done

In [1]:
from sre_parse import parse

parse('http[s]?')

[(LITERAL, 104), (LITERAL, 116), (LITERAL, 116), (LITERAL, 112), (MAX_REPEAT, (0, 1, [(LITERAL, 115)]))]

In [2]:
parse(b'a|(b|c)')

[(BRANCH, (None, [[(LITERAL, 97)], [(SUBPATTERN, (1, 0, 0, [(IN, [(LITERAL, 98), (LITERAL, 99)])]))]]))]

So we can use the standard library to parse regex patterns, giving nested lists of tag/value pairs.

Turning these back into strings is "a simple matter of code"... right?

In [3]:
import re
from hypothesis.strategies import from_regex

pattern = r'^http[s]?://\w+\.com$'
pattern = re.compile(pattern, re.ASCII)
strategy = from_regex(pattern)
strategy

from_regex(re.compile(r'^http[s]?://\w+\.com$', ))

In [4]:
for _ in range(20):
    print(strategy.example().strip())

https://wqCuqJ.com
https://5Ye4.com
http://PUZtaV230SYT.com
http://HwjdJ.com
http://GOD6.com
https://S.com
https://1.com
https://GVN0uM.com
http://p.com
http://AZ48q88y.com
https://bl.com
http://Rbyt.com
http://i_SL.com
https://Rr4.com
https://w.com
http://sf.com
https://0aJwey3KNPf.com
https://fK7Yc9iAfZ.com
http://vShs.com
http://gstccbSC331Pfal55Oow3OFF.com


In [5]:
strategy.wrapped_strategy

maybe_pad(regex=re.compile(r'^http[s]?://\w+\.com$', ), strategy=clear_cache_after_draw(tuples(just(''), just('http'), just('') | just('s'), just('://'), lists(elements=characters(whitelist_categories={'LC', 'Ll', 'Lm', 'Lo', 'Lt', 'Lu', 'Nd', 'Nl', 'No'}, blacklist_characters=set(), max_codepoint=127, whitelist_characters={'_'}), min_size=1).map(join), just('.com'), just('')).map(join)).filter(search), left_pad_strategy=just(''), right_pad_strategy=just('') | just('\n'))

In [6]:
from hypothesis import find

find(strategy, bool)

'http://0.com'

### Good news and bad news

Good: 

- it actually works!
- it's actually pretty useful!

~~Bad~~ *Potential upgrades*:

- complex strategies could do less work, if optimised in construction 
- some patterns break example shrinking

### Example shrinking?

In [7]:
from hypothesis import seed, given, strategies as st

@given(st.lists(st.floats(allow_nan=False)))
@seed(302934307671667531413257853548643485645)
def test_mean(xs):
    mean = sum(xs) / len(xs)
    assert min(xs) <= mean <= max(xs), mean

test_mean()

Falsifying example: test_mean(xs=[inf, -inf])
Traceback (most recent call last):
  File "c:\users\zac\documents\github\hypothesis-python\src\hypothesis\core.py", line 846, in run
    print_example=True, is_final=True
  File "c:\users\zac\documents\github\hypothesis-python\src\hypothesis\executors.py", line 58, in default_new_style_executor
    return function(data)
  File "c:\users\zac\documents\github\hypothesis-python\src\hypothesis\core.py", line 140, in run
    return test(*args, **kwargs)
  File "<ipython-input-7-01e6dee80f48>", line 4, in test_mean
    @seed(302934307671667531413257853548643485645)
  File "c:\users\zac\documents\github\hypothesis-python\src\hypothesis\core.py", line 591, in timed_test
    result = test(*args, **kwargs)
  File "<ipython-input-7-01e6dee80f48>", line 7, in test_mean
    assert min(xs) <= mean <= max(xs), mean
AssertionError: nan

Falsifying example: test_mean(xs=[])
Traceback (most recent call last):
  File "c:\users\zac\documents\github\hypothesis-

MultipleFailures: Hypothesis found 2 distinct failures.

`find(strategy, predicate)` returns the smallest example from a strategy such that `predicate(example)` is True.  For example,

In [8]:
find(from_regex('^[ab]*'), lambda s: True)

''

In [9]:
find(from_regex('^[ab]*'), lambda s: s.find('bb') >= 5)

'aaaaabb'

In [10]:
find(from_regex('a|'), lambda s: True)

'a'

Uh oh - the empty string would be a shorter match.  What happened here?

- The pattern `'a|'` was parsed to  
  `[(BRANCH, (None, [[(LITERAL, 97)], []]))]`
- This parse tree was interpreted as `sampled_from(['a', ''])`
- Hypothesis does not reorder branches

Could you just reorder branches before interpreting them?

### Mostly yes!

Reordering branches, by the shortest matching string, would be a great start.  

Better be careful about performance though, and we've "only" pushed there are still correctness issues lurking...

`find(from_regex('a+|(bb)*?'), bool)`

Reordering branches by shortest match will actually break this example!  The sequence of best matches goes  
`'', a, aa, bb, aaa, aaaa, bbbb, aaaaa, aaaaaa, bbbbbb, ...`

And putting 'zero or more of `'bb'`' before 'one or more `'a'`' gives the wrong answer for any predicate that requires a non-empty string.

### What else could we do?

The only way to be *fully correct* is to generate all possible strings, sort them, and sample from the list.  Unfortunately this is impractical in the general case [citation needed].

Some practical optimisations *might* include:

- Split repeaters into branches for zero, one, or more repeats
- 'lift' branches out of groups to make reordering more effective
- Split other constructs (eg sets into branches between ranges)
- 'fuse' nested constructs so branches can be lifted higher
- Talk to compiler experts and ask for suggestions!

The challenge is to balance performance, shrinking, and code complexity for real-world workloads.

Got some ideas?  Talk to me, vist the issue tracker, and get a `#Hacktoberfest` T-shirt!